# <center> <u>**iNaturalist** </u></center>





# Contexte

Il ya un certain nombre de problèmes et de conflits avec ces trois bdd

* inaturalist_2205 : Axel 
* inaturalist_2305 : Basile  
* inaturalist_2305_france : Basile 


On va tenter d'y répondre et de décrire les trois bases de données à disposition.

## 1. Photo_id pas unique entre les bases de données


**Par exemple :**

* data_bees_detection/whole_dataset/inaturalist_2305/Aglaoapis tridentata/82888774.jpg.
* data_bees_detection/whole_dataset/inaturalist_2205/Aglaoapis tridentata39788.jpg.

Qui est actuellement stockée à l'url [suivante](https://inaturalist-open-data.s3.amazonaws.com/photos/82888774/medium.jpg).

(Ce n'est pas un cas isolé)

**Deux hypothèes :**

* la manière ont a été enregistré inaturalist_2205 n'est pas espèces+photo_id
* les photo_id ont changé entre les deux téléchargements des csv de métadonnées

**Conclusion :**

* pour comparer les photos :
    * soit c'est à métadonnées égales
    * soit c'est on doit comparer les photos en elle même 

* pour updater la base de données locale avec les nouvelles photos, il faut garder une trace des photo_id 


## 2. Certaines photos sont sur inaturalist_2205 mais pas sur inaturalist_2305_france

**Par exemple :**

* data_bees_detection/whole_dataset/inaturalist_2205/Aglaopis tridentada/82888774.jpg

**Requète envoyée à inat.db :** 

```
Select latitude,longitude,observed_on from observations
Where observations.observation_uuid =
(Select observation_uuid from photos 
where photos.photo_id = 82888774);
```

**Résultat :** 

* Latitude : 50.2371548116
* Longitude : 14.8952773887
* Observed on : 2020-07-01

Après [vérif](https://www.google.com/maps/@50.2371548116,14.8952773887,12z?entry=ttu) c'est en République Tchèque. 

**Conclusion :** 

inaturalist_2205 n'est pas filtrée sur la France.

Quel filtre a été appliqué à inaturalist_2205 ?

A creuser ..


# 2. Caractérisation des bdd

## 2.1 Nombre de photos par BDD

In [ ]:
inaturalist_2305_france = !find "../../data_bees_detection/whole_dataset/inaturalist_2305_france" -type f | wc -l 
inaturalist_2305 = !find "../../data_bees_detection/whole_dataset/inaturalist_2305" -type f | wc -l
inaturalist_2205 = !find "../../data_bees_detection/whole_dataset/inaturalist_2205" -type f | wc -l

inaturalist_2205_2307 = int(inaturalist_2305_france[0])
inaturalist_2305 = int(inaturalist_2305[0])
inaturalist_2205 = int(inaturalist_2205[0])

import matplotlib.pyplot as plt

labels = '2305 : France', '2305 : World ', '2205 : World'
plt.figure(figsize=(10, 5))
plt.bar(labels, [inaturalist_2205_2307, inaturalist_2305, inaturalist_2205])
plt.title('Number of images per dataset')
plt.show()

: 

## 2.2 Nombre d'espèces par BDD

## 3. Provenance géographique des photos de chaque BDD

* inat_25_04 : Monde
* inaturalist_2305_france : France
* inaturalist_2205 : ??

Les csv sont trop volumineux, on est obligés de passer par une base de données pour les manipuler.

### 2.2.1 Tout migrer dans inat.db

#### 2.2.1.1 inaturalist_2205 >> inaturalist_2205

In [1]:
import os 
import re 
import pandas as pd

inaturalist_2205_path = '../../data_bees_detection/whole_dataset/inaturalist_2205'


extensions = []
for root,dirs,files in os.walk(inaturalist_2205_path):
    for file in files:
        
        
        extension = file.split('.')[-1]

        if extension not in extensions:
            extensions.append(extension)

print(extensions)

['jpeg', 'jpg', 'png', 'gif', 'txt']


In [2]:
# get the photo id out of the file names
# registered as genus speciesphoto_id.extension


def extract_genus_species(file_name):
    # Extract genus and species using regular expressions
    match = re.match(r"(\w+)\s([a-zA-Z]+)(\d+)", file_name)

    if match:
        genus = match.group(1)
        species = match.group(2)
        photo_id = match.group(3)
    else:
        return 'error','error'

    return genus+' '+species, photo_id 



dict_inaturalist_2205 = {}
for root,dirs,files in os.walk(inaturalist_2205_path):
    for file in files:
        
        genus_species,photo_id = extract_genus_species(file)
        dict_inaturalist_2205[photo_id] = genus_species

df_inaturalist_2205 = pd.DataFrame.from_dict(dict_inaturalist_2205, orient='index')

df_inaturalist_2205.reset_index(inplace=True)

df_inaturalist_2205.columns = ['photo_id', 'genus_species']

df_inaturalist_2205.head()




,photo_id,genus_species
0,74664,Hylaeus hyalinatus
1,45667,Hylaeus hyalinatus
2,34725,Hylaeus hyalinatus
3,36229,Hylaeus hyalinatus
4,33326,Hylaeus hyalinatus


Passage dans la base inat.db : 


In [ ]:
df_inaturalist_2205.to_csv('../../data_bees_detection/inat_utils/inaturalist_2205.csv',index=False)

Suite de commandes : 

```
cd '../.../data_bees_detection/inat_utils'
sqlite3 inat.db

CREATE TABLE inaturalist_2205 (
    photo_id INTEGER,
    taxon_name varying character(255),
);


CREATE UNIQUE INDEX "idx_inaturalist_2205_photo_id" ON "inaturalist_2205" (
    "photo_id"
);

CREATE INDEX "idx_inaturalist_2205_taxon_name" ON "inaturalist_2205" (
    "taxon_name"
);



.mode csv
.import .inaturalist_2205.csv inaturalist_2205
```

Est considérée commme ok une photo ayant meme photo_id et meme taxon_id dans inaturalist_2205 et inat_05_04.
(Comme vu précédemment ca ne concerne pas toutes les photos)




#### 2.2.2.1 inat_25_04 >> inaturalist_2305

In [4]:
import re 
import pandas as pd
import os 

dict = {}

for root,dirs,files in os.walk('../../data_bees_detection/whole_dataset/inat_25_04'):

    for file in files : 

        file_path = os.path.join(root,file)

        genus_species = file_path.split('/')[-2]
        photo_id = file_path.split('/')[-1].split('.')[0]

        dict[photo_id] = genus_species


df_inaturalist_2205 = pd.DataFrame.from_dict(dict, orient='index')

df_inaturalist_2205.reset_index(inplace=True)

df_inaturalist_2205.columns = ['photo_id', 'genus_species']

df_inaturalist_2205.head()

df_inaturalist_2205.to_csv('../../data_bees_detection/inat_utils/inaturalist_2305.csv',index=False)

Migrate to inat : mêmes commmande que précédemment

### 2.2.2 Requêter inat.db

SELECT o.latitude, o.longitude
FROM observations o
JOIN photos p ON o.observation_uuid = p.observation_uuid
JOIN inaturalist_2305 i1 ON p.photo_id = i1.photo_id
JOIN inaturalist_2205 i2 ON i1.photo_id = i2.photo_id AND i1.taxon_name = i2.name;
